In [44]:
import numpy as np
import random
from sys import exit
class MineField:
    def __init__(self):
        pass
    
    def new(self, size, mine_num, first=None): # size = w, h
        width, height = size
        self.field = np.empty((width, height),dtype=int)
        self.digged = []
        self.death = False
        self.quit = False
        
        nums = np.arange(width*height)
        
        if first is not None:
            nums = nums[~np.vectorize(lambda x: x in [w*height+h for w, h in self.adjacentcell(first)])(nums)]

        self.mines = [(point//height,point%height) for point in np.random.choice(nums, mine_num, replace=False)]

        for h in range(height):
            for w in range(width):
                self.field[w,h] = self.calcdigit((w,h))
        
    def draw(self, mine=False):
        width, height = self.field.shape
        to_zenkaku = dict(zip(range(9),'×１２３４５６７８'))
        
        print('　ＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ'[:width+1])
        for h in range(height):
            #if h < 20: print('①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳'[h],end='')
            #else: print(h+1,end='')
            print(f'{h+1:2}',end='')
            for w in range(width):
                if mine and ((w,h) in self.mines):
                    char = '＊'
                elif (w,h) in self.digged:
                    char = to_zenkaku[self.field[w,h]]
                else:
                    char = '・'
                print(char,end='')
            print('')
    
    def start(self, size=(10,10), mine_num=10, renew=True, renew_first_safe=True):       
        #if not renew: ignore renew_first_safe
        if renew:
            from functools import reduce
            fieldsize = reduce(lambda x,y: x*y, size)
            if not 0 < mine_num <= fieldsize - 9:
                print('Too many or few mines')
                return
            if renew_first_safe:
                t=MineField(); t.new(size=size, mine_num=mine_num); t.draw(); del t
                self.quit=False
                first = self.command(size)
                if self.quit: return
                self.new(size=size, mine_num=mine_num, first=first)
                self.autodig(first)
            else:
                self.new(size=size, mine_num=mine_num)
            
        while len(self.digged) < self.field.size - len(self.mines) and not self.death and not self.quit:
            self.draw()
            self.dig()
        if self.quit: return
        self.draw(True)
        return 'You lose' if self.death else 'You win!'
    
    def command(self, size):
        width, height = size
        while True:
            stri = input('command: ')
            if 'quit' in stri.lower():
                self.quit = True
                return None, None
            try:
                w=ord(stri[0].upper())-ord('A')
                h=int(''.join([ch for ch in stri if ch.isdigit()]))-1
                if w < width and h < height: break
            except: pass
            try: exec(stri)
            except: pass
            #(continue)
        return (w,h)
    
    def dig(self):
        w,h=self.command(self.field.shape)
        if self.quit: return
        self.autodig((w,h))

    def autodig(self, point):
        if point in self.digged: return
        if point in self.mines: self.death = True
        self.digged.append(point)
        if self.field[point] == 0: [self.autodig(adjs) for adjs in self.adjacentcell(point)]
                      
    def calcdigit(self, point):
        return len(set(self.adjacentcell(point)) & set(self.mines))
            
    def adjacentcell(self, point):
        width, height = self.field.shape
        w, h = point
        return [(x, y) for y in (h-1, h, h+1) for x in (w-1, w, w+1) if 0 <= x < width and 0 <= y < height]
    
    def undo(self):
        self.digged = self.digged[:-1]
        self.death = False
    

In [47]:
m = MineField()
m.start((10,10),91)

　ＡＢＣＤＥＦＧＨＩＪ
 1・・・・・・・・・・
 2・・・・・・・・・・
 3・・・・・・・・・・
 4・・・・・・・・・・
 5・・・・・・・・・・
 6・・・・・・・・・・
 7・・・・・・・・・・
 8・・・・・・・・・・
 9・・・・・・・・・・
10・・・・・・・・・・
command: d3
　ＡＢＣＤＥＦＧＨＩＪ
 1＊＊＊＊＊＊＊＊＊＊
 2＊＊５３５＊＊＊＊＊
 3＊＊３×３＊＊＊＊＊
 4＊＊５３５＊＊＊＊＊
 5＊＊＊＊＊＊＊＊＊＊
 6＊＊＊＊＊＊＊＊＊＊
 7＊＊＊＊＊＊＊＊＊＊
 8＊＊＊＊＊＊＊＊＊＊
 9＊＊＊＊＊＊＊＊＊＊
10＊＊＊＊＊＊＊＊＊＊


'You win!'